In [26]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Setup RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable both color (RGB) and depth streams
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start the pipeline
pipeline.start(config)

# Wait for the first valid frame
frames = pipeline.wait_for_frames()

# Get color and depth frames
color_frame = frames.get_color_frame()
depth_frame = frames.get_depth_frame()

# Convert to numpy arrays
color_image = np.asanyarray(color_frame.get_data())
depth_image = np.asanyarray(depth_frame.get_data())

# Get the color camera intrinsics
color_intrinsics = color_frame.profile.as_video_stream_profile().get_intrinsics()

# Print the intrinsic parameters
print("Color Intrinsics:")
print(f"Width: {color_intrinsics.width}")
print(f"Height: {color_intrinsics.height}")
print(f"Focal Length: (fx={color_intrinsics.fx}, fy={color_intrinsics.fy})")
print(f"Principal Point: (cx={color_intrinsics.ppx}, cy={color_intrinsics.ppy})")

# Get the depth camera intrinsics (you can use the depth stream's intrinsics similarly)
depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()

print("\nDepth Intrinsics:")
print(f"Width: {depth_intrinsics.width}")
print(f"Height: {depth_intrinsics.height}")
print(f"Focal Length: (fx={depth_intrinsics.fx}, fy={depth_intrinsics.fy})")
print(f"Principal Point: (cx={depth_intrinsics.ppx}, cy={depth_intrinsics.ppy})")

# Define chessboard pattern size (number of inner corners, e.g., 9x6 chessboard)
pattern_size = (8, 6)  # 9 columns, 6 rows of inner corners

# Convert the color image to grayscale
gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

# Find chessboard corners
ret, corners = cv2.findChessboardCorners(gray, pattern_size)

if ret:
    # Refine corner locations for subpixel accuracy
    cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                     criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))

    # Draw detected corners for visualization
    cv2.drawChessboardCorners(color_image, pattern_size, corners, ret)
    cv2.imshow('Detected Chessboard Corners', color_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Chessboard corners not detected.")


# Assuming we have the corners detected, let's map them to 3D world coordinates
world_points = []

for corner in corners:
    x, y = corner.ravel()

    # Get the depth value for the current pixel (in millimeters)
    depth = depth_frame.get_distance(int(x), int(y))*1000  # depth in meters
    if depth > 0:
        # Back-project the 2D pixel to 3D world coordinates
        world_point = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x, y], depth)
        world_points.append(world_point)

world_points = np.array(world_points)
print("3D world coordinates of corners:")
print(world_points)

# The 3D points (world coordinates) need to be converted to 2D coordinates in the world plane (Z=0)
# If the world points are in 3D (x, y, z), let's consider the x and y values (flattening the Z dimension)

# Convert 3D world points to 2D (assuming Z=0)
world_points_2d = world_points[:, :2]  # Only use the x and y coordinates

# Now compute the homography matrix between the 2D image points and 2D world points
H, _ = cv2.findHomography(world_points_2d, corners.reshape(-1, 2))

print("Homography Matrix H:")
print(H)





# Show the color image
cv2.imshow('Color Image', color_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

pipeline.stop()


Color Intrinsics:
Width: 640
Height: 480
Focal Length: (fx=607.0476684570312, fy=606.38427734375)
Principal Point: (cx=317.7261962890625, cy=230.09971618652344)

Depth Intrinsics:
Width: 640
Height: 480
Focal Length: (fx=589.3923950195312, fy=589.3923950195312)
Principal Point: (cx=314.98797607421875, cy=242.5380859375)
3D world coordinates of corners:
[[  43.993927     40.08396149  983.00006104]
 [  72.78414917   40.37263107  985.00006104]
 [ 101.92642975   40.84933853  985.00006104]
 [ 130.88581848   41.21699142  984.        ]
 [ 160.27853394   41.70043945  989.        ]
 [ 189.5683136    41.92135239  990.00006104]
 [ 221.3722229    42.56097412 1000.        ]
 [ 251.64576721   42.86922836 1003.        ]
 [  43.79205704   68.75491333  982.00006104]
 [  72.61325836   69.39443207  983.00006104]
 [ 101.42003632   69.77030945  984.        ]
 [ 130.92440796   70.16841125  986.00006104]
 [ 159.91317749   70.25945282  987.00006104]
 [ 189.54402161   70.8744812   990.00006104]
 [ 219.1191864 

In [40]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Setup RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable both color (RGB) and depth streams
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start the pipeline
pipeline.start(config)

# Wait for the first valid frame
frames = pipeline.wait_for_frames()

# Get color and depth frames
color_frame = frames.get_color_frame()
depth_frame = frames.get_depth_frame()

# Convert to numpy arrays
color_image = np.asanyarray(color_frame.get_data())
depth_image = np.asanyarray(depth_frame.get_data())

# Get the color camera intrinsics
color_intrinsics = color_frame.profile.as_video_stream_profile().get_intrinsics()

# Get the depth camera intrinsics
depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()

# Define chessboard pattern size (number of inner corners, e.g., 9x6 chessboard)
pattern_size = (8, 6)  # 9 columns, 6 rows of inner corners

# Define the real-world square size (in meters, e.g., 20 mm = 0.02 m)
square_size = 0.028*1000  # 20mm per square

# Threshold for the reprojection error (in pixels)
reprojection_error_threshold = 0.5  # You can adjust this threshold

# Function to compute the reprojection error
def compute_reprojection_error(image_points, world_points, H):
    """
    Computes the mean reprojection error between the image points and
    the points reprojected from the world points using the homography matrix H.
    """
    projected_points = []
    
    for point in world_points:
        world_point_2d = np.array([point[0], point[1], 1.0])  # Homogeneous coordinates
        projected_point = H.dot(world_point_2d)
        projected_point /= projected_point[2]  # Normalize by z to get the 2D point
        
        projected_points.append(projected_point[:2])  # Take only x, y

    projected_points = np.array(projected_points)

    # Calculate the Euclidean distance between the projected points and the actual image points
    error = np.sqrt(np.sum((projected_points - image_points) ** 2, axis=1))
    
    # Return the mean reprojection error
    return np.mean(error)

# Loop for calibration until the error is below the threshold
iteration = 0
while True:
    # Convert the color image to grayscale
    gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size)

    if ret:
        # Refine corner locations for subpixel accuracy
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                         criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))

        # Draw detected corners for visualization
        cv2.drawChessboardCorners(color_image, pattern_size, corners, ret)
        cv2.imshow('Detected Chessboard Corners', color_image)
        cv2.waitKey(0)

        # Map the corners to 3D world coordinates
        world_points = []

        for row in range(pattern_size[1]):  # For each row in the checkerboard
            for col in range(pattern_size[0]):  # For each column in the checkerboard
                # The world coordinates of each corner based on its grid position
                world_point = np.array([col * square_size, row * square_size, 0])  # Z=0 for the checkerboard plane
                world_points.append(world_point)

        world_points = np.array(world_points)

        # Convert 3D world points to 2D (Z=0) by ignoring the z-coordinate
        world_points_2d = world_points[:, :2]  # Only use x and y for 2D homography

        # Compute the homography matrix between the 2D image points and 2D world points
        H, _ = cv2.findHomography(world_points_2d, corners.reshape(-1, 2))

        # Compute the reprojection error
        reprojection_error = compute_reprojection_error(corners.reshape(-1, 2), world_points_2d, H)

        print(f"Iteration {iteration} - Reprojection Error: {reprojection_error}")

        # Check if the reprojection error is below the threshold
        if reprojection_error < reprojection_error_threshold:
            print("Calibration complete - Reprojection error is below threshold.")
            break

        # Increment the iteration counter
        iteration += 1

    else:
        print("Chessboard corners not detected.")

    # Stop the pipeline after the calibration is complete
    pipeline.stop()

# Final Homography matrix
print("Final Homography Matrix H:")
print(H)

# Show the color image
cv2.imshow('Color Image', color_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Iteration 0 - Reprojection Error: 0.15258733266474747
Calibration complete - Reprojection error is below threshold.
Final Homography Matrix H:
[[ 6.09542798e-01 -1.67279102e-02  2.93828535e+02]
 [-5.70913915e-03  5.98333405e-01  2.25283837e+02]
 [-1.39341271e-05 -5.77484196e-05  1.00000000e+00]]


In [47]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Setup RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable both color (RGB) and depth streams
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start the pipeline
pipeline.start(config)

# Wait for the first valid frame
frames = pipeline.wait_for_frames()

# Get color and depth frames
color_frame = frames.get_color_frame()
depth_frame = frames.get_depth_frame()

# Convert to numpy arrays
color_image = np.asanyarray(color_frame.get_data())
depth_image = np.asanyarray(depth_frame.get_data())

# Get the color camera intrinsics
color_intrinsics = color_frame.profile.as_video_stream_profile().get_intrinsics()

# Get the depth camera intrinsics
depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()


# Get the size of the original image
height, width = color_image.shape[:2]

# Optionally, you can compute the bounding box of the transformed image, to avoid clipping
corners = np.array([[0, 0], [width-1, 0], [0, height-1], [width-1, height-1]], dtype=np.float32)
transformed_corners = cv2.perspectiveTransform(corners[None, :, :], H)

# Get the bounding box of the transformed corners
min_x, min_y = np.min(transformed_corners, axis=1).ravel()
max_x, max_y = np.max(transformed_corners, axis=1).ravel()

# Compute the size of the output image
output_width = int(max_x - min_x)
output_height = int(max_y - min_y)

# Translate the homography to ensure the transformed image is properly placed
translation_matrix = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]])
adjusted_H = translation_matrix @ H

# Apply the adjusted homography to get the top-down view
depth_image = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.25), cv2.COLORMAP_VIRIDIS)
top_down_color = cv2.warpPerspective(color_image, adjusted_H, (output_width, output_height))
top_down_depth = cv2.warpPerspective(depth_image, adjusted_H, (output_width, output_height))


scale_factor = 1.5
top_down_color = cv2.resize(top_down_color,None,fx =  scale_factor,fy = scale_factor)
 
top_down_depth = cv2.resize(top_down_depth,None,fx =  scale_factor,fy = scale_factor)
# Show the top-down images
cv2.imshow('Original Frae',color_image )
cv2.imshow("Top-Down Color Image", top_down_color)
cv2.imshow("Top-Down Depth Image", top_down_depth)

# Wait for a key press before closing the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


Transforation Code


In [ ]:
get_H = False
if get_H == True:


    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, framerate = 30)
    config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, framerate = 30)

    #create align object
    align_to = rs.stream.color
    align = rs.align(align_to)

    # Start the pipeline
    pipeline.start(config)

    # Wait for the first valid frame
    frames = pipeline.wait_for_frames()

    # Get color and depth frames
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()

    # Convert to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Get the color camera intrinsics
    color_intrinsics = color_frame.profile.as_video_stream_profile().get_intrinsics()

    # Get the depth camera intrinsics
    depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()

    # Define chessboard pattern size (number of inner corners, e.g., 9x6 chessboard)
    pattern_size = (8, 6)  # 9 columns, 6 rows of inner corners

    # Define the real-world square size (in meters, e.g., 20 mm = 0.02 m)
    square_size = 0.028*1000  # 20mm per square

    # Threshold for the reprojection error (in pixels)
    reprojection_error_threshold = 0.5  # You can adjust this threshold

    # Function to compute the reprojection error
    def compute_reprojection_error(image_points, world_points, H):
        """
        Computes the mean reprojection error between the image points and
        the points reprojected from the world points using the homography matrix H.
        """
        projected_points = []
        
        for point in world_points:
            world_point_2d = np.array([point[0], point[1], 1.0])  # Homogeneous coordinates
            projected_point = H.dot(world_point_2d)
            projected_point /= projected_point[2]  # Normalize by z to get the 2D point
            
            projected_points.append(projected_point[:2])  # Take only x, y

        projected_points = np.array(projected_points)

        # Calculate the Euclidean distance between the projected points and the actual image points
        error = np.sqrt(np.sum((projected_points - image_points) ** 2, axis=1))
        
        # Return the mean reprojection error
        return np.mean(error)

    # Loop for calibration until the error is below the threshold
    iteration = 0
    while True:
        # Convert the color image to grayscale
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, pattern_size)

        if ret:
            # Refine corner locations for subpixel accuracy
            cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))

            # Draw detected corners for visualization
            cv2.drawChessboardCorners(color_image, pattern_size, corners, ret)
            cv2.imshow('Detected Chessboard Corners', color_image)
            cv2.waitKey(0)

            # Map the corners to 3D world coordinates
            world_points = []

            for row in range(pattern_size[1]):  # For each row in the checkerboard
                for col in range(pattern_size[0]):  # For each column in the checkerboard
                    # The world coordinates of each corner based on its grid position
                    world_point = np.array([col * square_size, row * square_size, 0])  # Z=0 for the checkerboard plane
                    world_points.append(world_point)

            world_points = np.array(world_points)

            # Convert 3D world points to 2D (Z=0) by ignoring the z-coordinate
            world_points_2d = world_points[:, :2]  # Only use x and y for 2D homography

            # Compute the homography matrix between the 2D image points and 2D world points
            H, _ = cv2.findHomography(world_points_2d, corners.reshape(-1, 2))

            # Compute the reprojection error
            reprojection_error = compute_reprojection_error(corners.reshape(-1, 2), world_points_2d, H)

            print(f"Iteration {iteration} - Reprojection Error: {reprojection_error}")

            # Check if the reprojection error is below the threshold
            if reprojection_error < reprojection_error_threshold:
                print("Calibration complete - Reprojection error is below threshold.")
                break

            # Increment the iteration counter
            iteration += 1

        else:
            print("Chessboard corners not detected.")

        # Stop the pipeline after the calibration is complete
        pipeline.stop()

    # Final Homography matrix
    print("Final Homography Matrix H:")
    print(H)

    # Show the color image
    cv2.imshow('Color Image', color_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_top_down_view(color_image, depth_image, H):
    # Get the color camera intrinsics
    #color_intrinsics = color_frame.profile.as_video_stream_profile().get_intrinsics()

    # Get the depth camera intrinsics
    #depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()


    # Get the size of the original image
    height, width = color_image.shape[:2]

    # Optionally, you can compute the bounding box of the transformed image, to avoid clipping
    corners = np.array([[0, 0], [width-1, 0], [0, height-1], [width-1, height-1]], dtype=np.float32)
    transformed_corners = cv2.perspectiveTransform(corners[None, :, :], H)

    # Get the bounding box of the transformed corners
    min_x, min_y = np.min(transformed_corners, axis=1).ravel()
    max_x, max_y = np.max(transformed_corners, axis=1).ravel()

    # Compute the size of the output image
    output_width = int(max_x - min_x)
    output_height = int(max_y - min_y)

    # Translate the homography to ensure the transformed image is properly placed
    translation_matrix = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]])
    adjusted_H = translation_matrix @ H

    # Apply the adjusted homography to get the top-down view
    depth_image_color_map = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.25), cv2.COLORMAP_VIRIDIS)
    top_down_color = cv2.warpPerspective(color_image, adjusted_H, (output_width, output_height))
    top_down_depth = cv2.warpPerspective(depth_image, adjusted_H, (output_width, output_height))


    scale_factor = 0.5
    top_down_color = cv2.resize(top_down_color,None,fx =  scale_factor,fy = scale_factor)
    
    top_down_depth = cv2.resize(top_down_depth,None,fx =  scale_factor,fy = scale_factor)
    # Show the top-down images
   
    #cv2.imshow("Top-Down Color Image", top_down_color)
    #cv2.imshow("Top-Down Depth Image", top_down_depth)
    return top_down_color,top_down_depth

In [ ]:
#Improve Depth Image

if get_H == True:
                masked_color_image_top_down,cropped_image_top_down, hull,box,box_detected = cut_region_between_hulls(top_down_depth,top_down_color,min_depth =0,max_depth = cutting_depth, factor= 0.12, cut_rect= True, improved_bounding_box= False)
            #box_detected = False
if get_H == True:
    top_down_color, top_down_depth = get_top_down_view(color_image, depth_image, H)

if get_H == True:
    if masked_color_image_top_down is not None and   isinstance(masked_color_image_top_down, np.ndarray):
        print(masked_color_image_top_down)
        resized_masked_image = cv2.resize(masked_color_image_top_down,None,fx =  scale_factor,fy = scale_factor)
        # Create the top horizontal stack
        
        cv2.imshow('Color Images top down', resized_masked_image)
    else:
        cv2.imshow('Color Images top down', resized_masked_image)



NameError: name 'get_H' is not defined